In [6]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error


In [2]:
pred_gpt = pd.read_csv(f'../generated/GPT3.5-headlines.csv', index_col=0) 
pred_non = pd.read_csv(f'../generated/Without-sentiment.csv', index_col=0)
pred_fb = pd.read_csv(f'../generated/finBERT-headlines.csv', index_col=0)
pred_llm = pd.read_csv(f'../generated/pred_llama3_70b.csv', index_col=0) 

In [7]:
mean_squared_error(y_true = pred_llm['Actual'], y_pred = pred_llm['Predicted'])

0.0041745700067604

In [8]:
mean_squared_error(y_true = pred_non['Actual'], y_pred = pred_non['Predicted'])

0.0041881887506367245

In [9]:
mean_squared_error(y_true = pred_fb['Actual'], y_pred = pred_fb['Predicted'])

0.00426675622420052

In [10]:
mean_squared_error(y_true = pred_gpt['Actual'], y_pred = pred_gpt['Predicted'])

0.004207293013247789